In [1]:
import sys
sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose')
sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src')
sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src/policies')

In [2]:
from policies.policies import fitted_q, model_smoothed_fitted_q
from Glucose import Glucose
from transition_estimation import MultivariateLinear
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [3]:
env = Glucose(horizon=30)
gamma = 0.9
number_of_value_iterations = 1
number_of_replicates = 30

## Baseline

In [4]:
scores = []
for rep in range(number_of_replicates):
    env.reset()
    done = False
    episode_score = 0.0
    while not done:
        action = np.random.binomial(1, 0.3)
        _, r, done = env.step(action)
        episode_score += r
    scores.append(episode_score)
print('mean score: {} se: {}'.format(np.mean(scores), np.std(scores) / np.sqrt(number_of_replicates)))

mean score: -136.9889663848336 se: 18.081104181510334


## Vanilla fitted Q

In [5]:
scores = []
for rep in range(number_of_replicates):
    env.reset()
    done = False
    episode_score = 0.0
    while not done:
        action = fitted_q(env, gamma, RandomForestRegressor, number_of_value_iterations=number_of_value_iterations)
        _, r, done = env.step(action)
        episode_score += r
    scores.append(episode_score)
print('mean score: {} se: {}'.format(np.mean(scores), np.std(scores) / np.sqrt(number_of_replicates)))

mean score: -59.64137765504188 se: 3.105193833989782


## Fitted Q with model smoothing (using kde objective)

In [6]:
transition_model_fitter = MultivariateLinear

In [7]:
scores = []
for rep in range(number_of_replicates):
    env.reset()
    done = False
    episode_score = 0.0
    while not done:
        action, pairwise_kernel_, kernel_sums = \
            model_smoothed_fitted_q(env, gamma, RandomForestRegressor, number_of_value_iterations, transition_model_fitter,
                                    pairwise_kernels_=None, kernel_sums=None)
        _, r, done = env.step(action)
        episode_score += r
    scores.append(episode_score)
print('mean score: {} se: {}'.format(np.mean(scores), np.std(scores) / np.sqrt(number_of_replicates)))

mean score: -65.02223944339298 se: 4.180683509383638


## Fitted Q with model smoothing (using estimated MSE-optimal combination)